https://stevenloria.com/simple-text-classification/

In [1]:
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
import random
import time
import os
import sys
import os.path
import string

# ---- for TF-IDF & NLTK
import math
from textblob import TextBlob as tb
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from pathlib import Path
from beautifultable import BeautifulTable
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

import re, string, unicodedata
import contractions
import inflect
from bs4 import BeautifulSoup
from tabulate import tabulate

allRunsAccuracy = []

In [2]:
# REMOVE THE NL VALUES FROM THE FILE!

iFile = open(r"C:\Users\ani\Desktop\Course data Thesis\INTENT MINING\allMerged.txt", 'r', encoding = "ISO-8859-1")
sentences = iFile.read().split('\n')

with open(r"C:\Users\ani\Desktop\Course data Thesis\INTENT MINING\allMergedNoNL.csv", 'w', encoding = "ISO-8859-1") as oFile:
    for sent in sentences:
        parts = sent.split("|")
        if len(parts) == 2:
            label = parts[0]
            sentence = parts[1]

            if label == "NL": 
                pass
            else: 
                oFile.write(label+"|"+sentence+"\n")
            
iFile.close()

#### Pre-processing

In [3]:
iFilePath = r"C:\Users\ani\Dropbox\Stuff\university\data\250PerClass.csv"

fullSWList = ['about', 'above', 'after', 'again', 'against', 'all', 'a' , 'am', 'an', 'and', 'any', 'are', 'as', 
                'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'did', 
                'do', 'does', 'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 
                'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 
                'in', 'into', 'is', 'it', 'it', 'its', 'itself', 'just', "'ll", "'m", 'me', 'more', 'most', 'my', 
                'myself', 'no', 'nor', 'not', 'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours',
                'ourselves', 'out', 'over', 'own', "'re", 's', "'s", 'same', 'she', 'should', 'so', 'some', 'such',
                't', 'than', 'that', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 
                'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 
                'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'with', 
                'you', 'your', 'yours', 'yourself', 'yourselves','yeah','so','well']

UNallowedPunct = {".",",","[", "]", "=", "*", "/", "+", "-", "%", "#", "?", "(", ")", "-", "_", 
                    ";", ":", "'", "\"", "^", "`"}.union(string.ascii_lowercase)

def rmSW(sentenceList):
    print("[Removing punctuation from file..] ")
    oSentList = []
          
    sungleNum = re.compile("[0-9]+")
    biggerNum = re.compile("(\d+.\d+)")
    singleLetter = re.compile("(?i)(?<![a-z])[a-z](?![a-z])")
    
   
    for sent in sentenceList:
        parts = sent.split("|")
        if len(parts) == 2:
            sent = parts[1]
            label = parts[0]
            sentToTok = word_tokenize(sent)
            line = ""

            for tok in sentToTok:
                if tok in UNallowedPunct: pass
                elif tok in fullSWList: pass
                elif len(tok) == 1: pass
                elif sungleNum.match(tok) or biggerNum.match(tok) or singleLetter.match(tok): pass
                elif tok == "OMITTED" or tok == "NUMBER": pass
                else: line += tok+" "
            oSentList.append(label+"|"+line)
    
    return oSentList

#------------------------------------------------------

def stripMultiSpace(sentenceList):
    print("[Removing MultiSpace..] ")
    oSentList = []
    
    doubleSpaceCommaPattern = r'\s,\s'
    singleSpaceBothSidesCommaPat = r'(\s\,\s)'
    apostrophePat = r'\s\''
    finalDotPat = r'\s\.'
    doubleSpacePat = r'\s\s'

    for sent in sentenceList:
        sent = re.sub(doubleSpaceCommaPattern, ",", sent)
        sent = re.sub(singleSpaceBothSidesCommaPat, ",", sent)
        sent = re.sub(apostrophePat, "'", sent)
        sent = re.sub(finalDotPat, ".", sent)
        sent = re.sub(doubleSpacePat, " ", sent)
        oSentList.append(sent)
            
    return oSentList
        
#------------------------------------------------------    

def stem(sentenceList):
    print("[Stemming..]")
    stemmer = LancasterStemmer()
    oSentList = []
    
    for sent in sentenceList:
        parts = sent.split("|")
        sent = parts[1]
        label = parts[0]
        sentToTok = word_tokenize(sent)
        line = ""
        
        for tok in sentToTok:
            stem = stemmer.stem(tok)
            line += stem+" "
            
        oSentList.append(label+"|"+line+"\n")
            
    return oSentList  

#------------------------------------------------------    

def main():
    curFile = open(iFilePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK

    sentences = curFile.read().split("\n")
    #print(sentences)
    sentences = rmSW(sentences)
    sentences = stripMultiSpace(sentences)
    sentences = stem(sentences)
    
    oFilePath = r"C:\Users\ani\Dropbox\Stuff\university\data\processedMerged250PerClass.txt"
    with open(oFilePath, 'w', encoding = "utf-8") as oFile:
        print("[Writing to file..]",oFilePath)
        for sent in sentences:
            parts = sent.split("|")
            if len(parts) == 2:
                sent = parts[1]
                label = parts[0]
                if label == "NL":
                    pass
                else:
                    oFile.write(label+"|"+sent)

    curFile.close()
    
#---------------- 

main()

[Removing punctuation from file..] 
[Removing MultiSpace..] 
[Stemming..]
[Writing to file..] C:\Users\ani\Dropbox\Stuff\university\data\processedMerged250PerClass.txt


#### Split         [TRAIN:TEST]       [75:25]

In [4]:
file = open(r"C:\Users\ani\Dropbox\Stuff\university\data\processedMerged250PerClass.txt", 'r', encoding = "ISO-8859-1")
sentences = file.read().split('\n')
text = ""

for sent in sentences:
    text += sent+"\n"

sentences = text.split("\n")

trainS = []
testS = []
train = []
test = []

def split_testTrain():
    nrsplit = 0
    textsize = len(sentences)
    trainTestRatio = round(0.75 * textsize)

    #print("Total sent: ",textsize, " | TRAIN sent: ",trainTestRatio," | TEST sent: ",textsize-trainTestRatio)

    while not len(sentences) == 0:
        nrsplit += 1
        if nrsplit <= trainTestRatio:
            curLine = random.choice(sentences)
            sentences.remove(curLine)
            trainS.append(curLine) 
        elif nrsplit > trainTestRatio:
            curLine = random.choice(sentences)
            sentences.remove(curLine)
            testS.append(curLine) 

    #print("Sent: ",len(sentences)," | Train: ",len(trainS)," | Test:",len(testS))   
    print("TRAIN: ",len(trainS)," | TEST:",len(testS))   
    
    for sent in trainS:
        parts = sent.split('|')
        if len(parts) > 1:
            label = parts[0]
            sent = parts[1]
            tup = (sent,label)
            train.append(tup)
        else:
            continue

    for sent in testS:
        parts = sent.split('|')
        if len(parts) > 1:
            label = parts[0]
            sent = parts[1]
            tup = (sent,label)
            test.append(tup)
        else:
            continue
### ------------------ END SPLITING TEST / TRAIN with my own data
            
#split_testTrain()
#print("\n",train[:10])

### Takes about 5 min to train and output result PER ITERATION!!!!

In [5]:
### ------------------ Naive Bayes train
start = time.time()
runs = 1

with open(r"C:\Users\ani\Desktop\Course data Thesis\INTENT MINING\NBayeslogNew.txt", "a", encoding = "utf-8") as oFile:
    """ -------- ATTENTION: Nr of Iterations x 5 MIN!!! -------------- """
    while(runs <= 1):  
        startIter = time.time()
        split_testTrain()           #redo the train-test split
        
        bayes = NaiveBayesClassifier(train)        # TIME CONSUMING!!! TRAINING PHASE!

        # Compute accuracyf
        accuracy = bayes.accuracy(test)*100
        allRunsAccuracy.append(accuracy)
        oFile.write("\nAccuracy: {0:.2f} %".format(accuracy))
        print("\nAccuracy: {0:.2f} %".format(bayes.accuracy(test)*100))
        
        # Show 5 most informative features
        # This cannot be saved in the output file as it return None
        bayes.show_informative_features(5)
        runs += 1
        endIter = time.time()
        print("iteration execution time: {0:.2f} min".format((endIter - startIter)/60))
        
end = time.time()
print("Runs: ",len(allRunsAccuracy))
print("Execution time: {0:.2f} min".format((end - start)/60))

TRAIN:  939  | TEST: 313

Accuracy: 57.19 %
Most Informative Features
        contains(second) = True               CD : SM     =     30.3 : 1.0
           contains(bal) = True               CD : SM     =     29.1 : 1.0
        contains(exampl) = True               EX : CD     =     23.6 : 1.0
           contains(let) = True               EX : AP     =     16.4 : 1.0
            contains(av) = True               CD : SM     =     14.4 : 1.0
iteration execution time: 0.17 min
Runs:  1
Execution time: 0.17 min


In [6]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, train, test, cv=5)
print(scores)

NameError: name 'svm' is not defined

#### Test

In [ ]:
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
start = time.time()

train = [
    ('I love this sandwich.', 'pos'),
    ('This is an amazing place!', 'pos'),
    ('I feel very good about these beers.', 'pos'),
    ('This is my best work.', 'pos'),
    ("What an awesome view", 'pos'),
    ('I do not like this restaurant', 'neg'),
    ('I am tired of this stuff.', 'neg'),
    ("I can't deal with this", 'neg'),
    ('He is my sworn enemy!', 'neg'),
    ('My boss is horrible.', 'neg')
]
test = [
    ('The beer was good.', 'pos'),
    ('I do not enjoy my job', 'neg'),
    ("I ain't feeling dandy today.", 'neg'),
    ("I feel amazing!", 'pos'),
    ('Gary is a friend of mine.', 'pos'),
    ("I can't believe I'm doing this.", 'neg')
]

with open(r"C:\Users\ani\Desktop\Course data Thesis\NBayeslog.txt","a",encoding = "utf-8") as oFile:
    cl = NaiveBayesClassifier(train)
    
    """
    # Classify some text
    print("The other notion is existential stability,which means there exists the flipped it's called an existential quantifier   ==>  ",bayes.classify("The other notion is existential stability,which means there exists the flipped it's called an existential quantifier "))  # "pos"
    print("In fact,like saw,we looked at a counter example that allowed us to actually destabilize,which means drive the state off to infinity,by being unlucky or unclever in how we transitioned in and out of the different modes    ==>> ",bayes.classify("In fact,like saw,we looked at a counter example that allowed us to actually destabilize,which means drive the state off to infinity,by being unlucky or unclever in how we transitioned in and out of the different modes "))   # "neg"

    # Classify a TextBlob
    blob = TextBlob("And in the next lecture,we will see,first of all,why the tortoise is able to beat the hare and what to do about it,meaning how do you make the rabbit overtake the turtle  "
                    "We know what the solution to this system is", classifier=bayes)
    print(blob)
    print(blob.classify())

    for sentence in blob.sentences:
        print(sentence)
        print(sentence.classify())

    """

    # Compute accuracy
    accuracy = cl.accuracy(test)*100
    oFile.write("Accuracy: {0:.2f} % \n".format(accuracy))
    print("Accuracy: {0:.2f} %".format(cl.accuracy(test)*100))

    # Show 5 most informative features
    cl.show_informative_features(5)
    
end = time.time()
print("Execution time: {0:.3f} sec | {1:.5f} min".format((end - start),(end - start)/60))